In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from keras import layers
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import tensorflow as tf
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,BatchNormalization
import os
from PIL import Image
import cv2


Data Gathering and Preprocessing

In [3]:
# create data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Define the data generator for the validation and test sets
val_test_datagen = ImageDataGenerator(rescale=1./255)

# fit the model on data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')
    
validation_generator = val_test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    
    for root, _, files in os.walk(DIR):
        for file in files:
            if file.endswith('.png'):
                PATH = os.path.join(root,file)

                img = read(PATH)

                img = cv2.resize(img, (RESIZE,RESIZE))

                IMG.append(np.array(img))
    return IMG

real_train = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train/Real',64))
fake_train = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Train/Fake',64))
real_test = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid/Real',64))
fake_test = np.array(Dataset_loader('/content/drive/MyDrive/Group_Project_Data 2/Group_Project_Data/Valid/Fake',64))

Split dataset

In [8]:
# Create labels
real_train_label = np.ones(len(real_train))
fake_train_label = np.zeros(len(fake_train))
real_test_label = np.ones(len(real_test))
fake_test_label = np.zeros(len(fake_test))

# Merge data 
X_train = np.concatenate((real_train, fake_train), axis = 0)
Y_train = np.concatenate((real_train_label, fake_train_label), axis = 0)
X_test = np.concatenate((real_test, fake_test), axis = 0)
Y_test = np.concatenate((real_test_label, fake_test_label), axis = 0)

# Shuffle train data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]


Define Model Architecture

In [12]:
# define input shape
input_shape = (64, 64, 3)

# define model architecture
model = tf.keras.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5), # Adding Dropout layer with a rate of 0.5
    layers.Dense(1, activation='sigmoid')
])

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 62, 8)         224       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 31, 31, 8)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 16)        1168      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 12, 12, 32)        4640      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 6, 6, 32)        